# Curves

### CompositeCurve example

The first section here regards efficient operations and compositing two curves.

In [ ]:
from rateslib import dt, defaults
from rateslib.curves import Curve, LineCurve, CompositeCurve, MultiCsaCurve

In [ ]:
line_curve1 = LineCurve({dt(2022, 1, 1): 2.0, dt(2022, 1, 3): 4.0}, id="C1_")
line_curve2 = LineCurve({dt(2022, 1, 1): 0.5, dt(2022, 1, 3): 1.0}, id="C2_")
composite_curve = CompositeCurve(curves=(line_curve1, line_curve2))
composite_curve.rate(dt(2022, 1, 2))

In [ ]:
line_curve1._set_ad_order(1)
line_curve2._set_ad_order(1)
composite_curve.rate(dt(2022, 1, 2))

The code above demonstrates the summing of individual rates and of interoperability with Dual datatypes.

Below measures rate lookup.

In [ ]:
defaults.curve_caching = False

composite_curve = CompositeCurve(
    (
        Curve({dt(2022, 1, 1): 1.0, dt(2024, 1, 1): 0.95}, id="C1_"),
        Curve({dt(2022, 1, 1): 1.0, dt(2024, 1, 1): 0.99}, id="C2_"),
    )
)
%timeit composite_curve.rate(dt(2022, 6, 1), "1y")  

### MultiCsaCurve

In [ ]:
c1 = Curve({dt(2022, 1, 1): 1.0, dt(2052, 1, 1): 0.5})
c2 = Curve({dt(2022, 1, 1): 1.0, dt(2032, 1, 1): 0.4, dt(2052, 1, 1):0.39}) 
mcc = MultiCsaCurve([c1, c2])

%timeit c2[dt(2052, 1, 1)]

In [ ]:
%timeit mcc[dt(2052, 1, 1)]

### Error in approximated rates and execution time

In [ ]:
import numpy as np
MIN, MAX, SAMPLES, DAYS, d = 0, 4, 100000, 3, 1.0/365
c1 = np.random.rand(DAYS, SAMPLES) * (MAX - MIN) + MIN
c2 = np.random.rand(DAYS, SAMPLES) * (MAX - MIN) + MIN
r_true=((1 + d * (c1 + c2) / 100).prod(axis=0) - 1) * 100 / (d * DAYS)
c1_bar = ((1 + d * c1 / 100).prod(axis=0)**(1/DAYS) - 1) * 100 / d
c2_bar = ((1 + d * c2 / 100).prod(axis=0)**(1/DAYS) - 1) * 100 / d
r_bar = ((1 + d * (c1_bar + c2_bar) / 100) ** DAYS - 1) * 100 / (d * DAYS)
np.histogram(np.abs(r_true-r_bar), bins=[0, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1]) 

### Curve operations: shift

In [ ]:
curve = Curve({dt(2022, 1, 1): 1.0, dt(2023, 1, 1): 0.98}, convention="Act365F", id="v", ad=1)
curve.rate(dt(2022, 6, 1), "1b")

In [ ]:
shifted_curve = curve.shift(50)
shifted_curve.rate(dt(2022, 6, 1), "1b")

In [ ]:
type(shifted_curve)

In [ ]:
%timeit curve.rate(dt(2022, 6, 1), "1b")

In [ ]:
%timeit shifted_curve.rate(dt(2022, 6, 1), "1b")

### Curve operations: roll

In [ ]:
curve = Curve(
    nodes={dt(2022, 1, 1): 1.0, dt(2023, 1, 1): 0.98, dt(2024, 1, 1): 0.97},
    t=[dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1),
       dt(2023, 1, 1),
       dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1)]
)
print(curve.rate(dt(2022, 6, 1), "1d"))
print(curve.roll("30d").rate(dt(2022, 7, 1), "1d"))

In [ ]:
line_curve = LineCurve(
    nodes={dt(2022, 1, 1): 2.0, dt(2023, 1, 1): 2.6, dt(2024, 1, 1): 2.5},
    t=[dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1),
       dt(2023, 1, 1),
       dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1)]
)
print(line_curve.rate(dt(2022, 6, 1)))
print(line_curve.roll("-31d").rate(dt(2022, 5, 1), "1d"))

### Curve operations: translate

In [ ]:
for interpolation in [
    "linear", "log_linear", "linear_index", "flat_forward", "flat_backward", "linear_zero_rate"
]:
    curve = Curve(
        nodes={dt(2022, 1, 1): 1.0, dt(2022, 2, 1):0.998, dt(2022, 3, 1): 0.995}, 
        interpolation=interpolation
    )
    curve_translated = curve.translate(dt(2022, 1, 15)) 
    print(
        curve.rate(dt(2022, 2, 15), "1d"),
        curve_translated.rate(dt(2022, 2, 15), "1d") 
    )

### Operations on CompositeCurves

In [ ]:
composite_curve.rate(dt(2022, 6, 1), "1d")

In [ ]:
composite_curve.shift(50).rate(dt(2022, 6, 1), "1d")

In [ ]:
composite_curve.roll("30d").rate(dt(2022, 7, 1), "1d")

In [ ]:
composite_curve.translate(dt(2022, 5, 1)).rate(dt(2022, 6, 1), "1d")